In [7]:
import tkinter as tk
from tkinter import Button
import cv2
import numpy as np
import pytesseract
from PIL import Image, ImageTk
from facenet_pytorch import MTCNN
import os
import threading
import csv

class RegistroRostroPlacaApp:
    def __init__(self, root, mtcnn):
        self.root = root
        self.root.title("Registro de Rostro y Placa")

        self.mtcnn = mtcnn
        self.fotos_por_cliente = 20
        self.cliente_num = 1
        self.k = 0
        self.destino = 'clientes/'
        self.new_dim = (100, 100)
        self.Ctexto_rostro = ''
        self.Ctexto_placa = ''
        self.path_cliente = ''
        self.csv_filename = 'registros.csv'
        self.cliente = ''
        self.face_recognizer = None  # Se inicializará después de registrar rostros
        self.training_complete = False  # Indica si el reconocedor ha sido entrenado
        self.entrenar_reconocedor()
        self.create_widgets()

    def create_widgets(self):
        self.label = tk.Label(self.root, text="Presiona los botones para comenzar el registro de rostro y placa")
        self.label.pack(pady=10)

        self.registrar_rostro_boton = Button(self.root, text="Registrar Rostro", command=self.registrar_rostro)
        self.registrar_rostro_boton.pack(pady=10)

        self.registrar_placa_boton = Button(self.root, text="Registrar Placa", command=self.registrar_placa)
        self.registrar_placa_boton.pack(pady=10)

        self.reconocimiento_rostro_boton = Button(self.root, text="Reconocimiento de Rostro", command=self.reconocimiento_rostro)
        self.reconocimiento_rostro_boton.pack(pady=10)

    def registrar_rostro(self):
        if not self.training_complete:
            self.entrenar_reconocedor()  # Entrenar el reconocedor de rostros
            self.training_complete = True  # Indicar que el entrenamiento está completo

        if self.k == 0:
            self.registrar_rostro_boton.config(state=tk.DISABLED)
            self.cliente = f"cliente_{self.cliente_num}"
            path_cliente = self.destino + self.cliente

            while os.path.exists(path_cliente):
                self.cliente_num += 1
                self.cliente = f"cliente_{self.cliente_num}"
                path_cliente = self.destino + self.cliente

            print("Creando el directorio para: '{}' ".format(path_cliente))
            os.makedirs(path_cliente)

            self.path_cliente = path_cliente

            self.thread = threading.Thread(target=self.capturar_video_rostro, args=(path_cliente,))
            self.thread.start()

    def capturar_video_rostro(self, path_cliente):
        source = 0
        cam = cv2.VideoCapture(source)

        while self.k < self.fotos_por_cliente:
            ret, frame = cam.read()

            if not ret or frame is None:  # Verificar si la captura fue exitosa y frame no es None
                print("Error al capturar el fotograma.")
                break

            frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            boxes, confidence = self.mtcnn.detect(frame_pil)

            if np.ndim(boxes) != 0:
                box = boxes[0]
                c = confidence[0]
                box = box.astype(int)
                x, y, w, h = box

                if x > 0 and y > 0 and c > 0.95:
                    cv2.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
                    cv2.imshow('frame', frame)
                    print(f"Probabilidad rostro: {c}")
                    f_region = frame_pil.crop(box)
                    f_region = f_region.resize(self.new_dim)
                    new_name = path_cliente + '/img_' + str(self.k) + '.png'
                    self.k += 1
                    f_region.save(new_name)

            cv2.imshow('frame', frame)
            key = cv2.waitKey(1)
            if key == 27:
                break

        cv2.destroyAllWindows()
        self.label.config(text="Registro de rostro completado")
        self.registrar_rostro_boton.config(state=tk.NORMAL)
        self.k = 0

    # Agregar información al archivo CSV
        with open(self.csv_filename, 'a', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            row = [self.cliente] + [f'{self.path_cliente}/img_{i}.png' for i in range(1, self.fotos_por_cliente + 1)] + [self.Ctexto_rostro]
            csv_writer.writerow(row)

    def registrar_placa(self):
        if self.k == 0:
            self.registrar_placa_boton.config(state=tk.DISABLED)
            self.thread = threading.Thread(target=self.capturar_video_placa)
            self.thread.start()

    def capturar_video_placa(self):
        source = 0
        cap = cv2.VideoCapture(source)
        Ctexto = ''

        while True:
            ret, frame = cap.read()

            if ret is False:
                break

            al, an, c = frame.shape

            x1 = int(an / 3)
            x2 = int(x1 * 2)

            y1 = int(al / 3)
            y2 = int(y1 * 2)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            recorte = frame[y1:y2, x1:x2]

            mB = recorte[:, :, 0]
            mG = recorte[:, :, 1]
            mR = recorte[:, :, 2]

            Color = cv2.absdiff(mG, mB)

            _, umbral = cv2.threshold(Color, 40, 255, cv2.THRESH_BINARY)

            contornos, _ = cv2.findContours(umbral, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            contornos = sorted(contornos, key=lambda x: cv2.contourArea(x), reverse=True)

            for contorno in contornos:
                area = cv2.contourArea(contorno)
                if 500 < area < 5000:
                    x, y, ancho, alto = cv2.boundingRect(contorno)
                    xpi, ypi, xpf, ypf = x + x1, y + y1, x + ancho + x1, y + alto + y1

                    cv2.rectangle(frame, (xpi, ypi), (xpf, ypf), (255, 255, 0), 2)

                    placa = frame[ypi:ypf, xpi:xpf]
                    alp, anp, _ = placa.shape

                    Mva = np.zeros((alp, anp))

                    mBp = placa[:, :, 0]
                    mGp = placa[:, :, 1]
                    mRp = placa[:, :, 2]

                    for col in range(alp):
                        for fil in range(anp):
                            Max = max(mRp[col, fil], mGp[col, fil], mBp[col, fil])
                            Mva[col, fil] = 255 - Max

                    _, bin = cv2.threshold(Mva, 150, 255, cv2.THRESH_BINARY)

                    bin = bin.reshape(alp, anp)
                    bin = Image.fromarray(bin)
                    bin = bin.convert("L")

                    if 36 < alp < 82:
                        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
                        config = "--psm 1"
                        texto = pytesseract.image_to_string(bin, config=config)

                        if len(texto) >= 7:
                            Ctexto = texto

                    break

            cv2.putText(frame, 'Procesando placa', (x1 - 30, y1 + 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y1 + 220), (x2, y1 + 260), (0, 0, 0), cv2.FILLED)
            cv2.putText(frame, Ctexto[0:7], (x1 + 40, y1 + 250), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow("Registro de Placa", frame)

            t = cv2.waitKey(1)
            if t == ord('q') or t == 27:
                break

        cap.release()
        cv2.destroyAllWindows()
        self.label.config(text="Registro de placa completado")
        self.registrar_placa_boton.config(state=tk.NORMAL)

        # Agregar información al archivo CSV
        with open(self.csv_filename, 'a', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            row = [self.cliente] + [f'{self.path_cliente}/img_{i}.png' for i in range(1, self.fotos_por_cliente + 1)] + [self.Ctexto_placa]
            csv_writer.writerow(row)

    def entrenar_reconocedor(self):
        dataPath = self.destino
        peopleList = os.listdir(dataPath)
        labels = []
        facesData = []
        label = 0

        for nameDir in peopleList:
            personPath = dataPath + '/' + nameDir
            print('Leyendo las imágenes')

            for fileName in os.listdir(personPath):
                print('Rostros: ', nameDir + '/' + fileName)
                labels.append(label)
                img_path = personPath + '/' + fileName
                facesData.append(cv2.imread(img_path, 0))  # Utilizar la imagen leída
            label += 1

        if facesData and labels:  # Verificar que haya datos antes de entrenar
            self.face_recognizer = cv2.face.LBPHFaceRecognizer_create()
            self.face_recognizer.train(facesData, np.array(labels))
            self.face_recognizer.write('modeloLBPHFace.xml')
            print("Modelo almacenado...")
        else:
            print("No se encontraron datos para entrenar el modelo.")

            
    def reconocimiento_rostro(self):
        if not self.training_complete:
            print("Entrenando el reconocedor...")
            self.entrenar_reconocedor()
            self.training_complete = True

        cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

        while True:
            ret, frame = cap.read()
            if ret == False:
                break
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            auxFrame = gray.copy()

            faces = faceClassif.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                rostro = auxFrame[y:y+h, x:x+w]
                rostro = cv2.resize(rostro, (150, 150), interpolation=cv2.INTER_CUBIC)

                result = self.face_recognizer.predict(rostro)

                cv2.putText(frame, 'Cliente {}'.format(result[0] + 1), (x, y-5), 1, 1.3, (255, 255, 0), 1, cv2.LINE_AA)

                if result[1] < 70:
                    cv2.putText(frame, 'Reconocido: {}'.format(result[0] + 1), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv2.LINE_AA)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                else:
                    cv2.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

            cv2.imshow('Reconocimiento de Rostro', frame)
            k = cv2.waitKey(1)
            if k == 27:
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    device = 'cpu'
    mtcnn = MTCNN(keep_all=True, device=device)

    root = tk.Tk()
    app = RegistroRostroPlacaApp(root, mtcnn)
    root.mainloop()

print('\nDone')

Leyendo las imágenes
Rostros:  cliente_1/img_0.png
Rostros:  cliente_1/img_1.png
Rostros:  cliente_1/img_10.png
Rostros:  cliente_1/img_11.png
Rostros:  cliente_1/img_12.png
Rostros:  cliente_1/img_13.png
Rostros:  cliente_1/img_14.png
Rostros:  cliente_1/img_15.png
Rostros:  cliente_1/img_16.png
Rostros:  cliente_1/img_17.png
Rostros:  cliente_1/img_18.png
Rostros:  cliente_1/img_19.png
Rostros:  cliente_1/img_2.png
Rostros:  cliente_1/img_3.png
Rostros:  cliente_1/img_4.png
Rostros:  cliente_1/img_5.png
Rostros:  cliente_1/img_6.png
Rostros:  cliente_1/img_7.png
Rostros:  cliente_1/img_8.png
Rostros:  cliente_1/img_9.png
Modelo almacenado...
Entrenando el reconocedor...
Leyendo las imágenes
Rostros:  cliente_1/img_0.png
Rostros:  cliente_1/img_1.png
Rostros:  cliente_1/img_10.png
Rostros:  cliente_1/img_11.png
Rostros:  cliente_1/img_12.png
Rostros:  cliente_1/img_13.png
Rostros:  cliente_1/img_14.png
Rostros:  cliente_1/img_15.png
Rostros:  cliente_1/img_16.png
Rostros:  cliente_1/i